# Tutorial: Preprocessing Different File Types

- `FileTypeRouter`: Thành phần này sẽ giúp bạn định tuyến các tệp dựa trên loại MIME tương ứng của chúng đến các thành phần khác nhau
- `MarkdownToDocument`: Thành phần này sẽ giúp bạn chuyển đổi các file markdown thành Haystack Documents
- `PyPDFToDocument`: Thành phần này sẽ giúp bạn chuyển đổi file pdf thành Haystack Documents
- `TextFileToDocument`: Thành phần này sẽ giúp bạn chuyển đổi file văn bản thành Haystack Documents
- `DocumentJoiner`: Thành phần này sẽ giúp bạn nối các Tài liệu đến từ các nhánh khác nhau của đường ống
- `DocumentCleaner`(tùy chọn): Thành phần này sẽ giúp bạn làm cho Tài liệu dễ đọc hơn bằng cách xóa các khoảng trắng thừa, v.v.
- `DocumentSplitter`: Thành phần này sẽ giúp bạn chia Tài liệu của mình thành nhiều phần
- `SentenceTransformersDocumentEmbedder`: Thành phần này sẽ giúp bạn tạo các phần nhúng cho Tài liệu.
- `DocumentWriter`: Thành phần này sẽ giúp bạn ghi Documents vào DocumentStore

In [2]:
# %%bash
# pip install haystack-ai
# pip install "sentence-transformers>=2.2.0" "huggingface_hub>=0.22.0"
# pip install markdown-it-py mdit_plain pypdf
# pip install gdown


## Download All Files

In [3]:
import gdown

url = "https://drive.google.com/drive/folders/1n9yqq5Gl_HWfND5bTlrCwAOycMDt5EMj"
output_dir = "recipe_files"

gdown.download_folder(url, quiet=True, output=output_dir)


['recipe_files/vegan_flan_recipe.md',
 'recipe_files/vegan_keto_eggplant_recipe.pdf',
 'recipe_files/vegan_sunflower_hemp_cheese_recipe.txt']

## Create a Pipeline to Index Documents

In [4]:
from haystack.components.writers import DocumentWriter
from haystack.components.converters import MarkdownToDocument, PyPDFToDocument, TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()
file_type_router = FileTypeRouter(mime_types=["text/plain", "application/pdf", "text/markdown"])
text_file_converter = TextFileToDocument()
markdown_converter = MarkdownToDocument()
pdf_converter = PyPDFToDocument()
document_joiner = DocumentJoiner()


/Users/ngohongthai/miniconda3/envs/nmt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(split_by="word", split_length=150, split_overlap=50)

In [6]:
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store)

In [7]:
preprocessing_pipeline = Pipeline()
preprocessing_pipeline.add_component(instance=file_type_router, name="file_type_router")
preprocessing_pipeline.add_component(instance=text_file_converter, name="text_file_converter")
preprocessing_pipeline.add_component(instance=markdown_converter, name="markdown_converter")
preprocessing_pipeline.add_component(instance=pdf_converter, name="pypdf_converter")
preprocessing_pipeline.add_component(instance=document_joiner, name="document_joiner")
preprocessing_pipeline.add_component(instance=document_cleaner, name="document_cleaner")
preprocessing_pipeline.add_component(instance=document_splitter, name="document_splitter")
preprocessing_pipeline.add_component(instance=document_embedder, name="document_embedder")
preprocessing_pipeline.add_component(instance=document_writer, name="document_writer")

preprocessing_pipeline.connect("file_type_router.text/plain", "text_file_converter.sources")
preprocessing_pipeline.connect("file_type_router.application/pdf", "pypdf_converter.sources")
preprocessing_pipeline.connect("file_type_router.text/markdown", "markdown_converter.sources")
preprocessing_pipeline.connect("text_file_converter", "document_joiner")
preprocessing_pipeline.connect("pypdf_converter", "document_joiner")
preprocessing_pipeline.connect("markdown_converter", "document_joiner")
preprocessing_pipeline.connect("document_joiner", "document_cleaner")
preprocessing_pipeline.connect("document_cleaner", "document_splitter")
preprocessing_pipeline.connect("document_splitter", "document_embedder")
preprocessing_pipeline.connect("document_embedder", "document_writer")


🚅 Components
  - file_type_router: FileTypeRouter
  - text_file_converter: TextFileToDocument
  - markdown_converter: MarkdownToDocument
  - pypdf_converter: PyPDFToDocument
  - document_joiner: DocumentJoiner
  - document_cleaner: DocumentCleaner
  - document_splitter: DocumentSplitter
  - document_embedder: SentenceTransformersDocumentEmbedder
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.text/plain -> text_file_converter.sources (List[Path])
  - file_type_router.application/pdf -> pypdf_converter.sources (List[Path])
  - file_type_router.text/markdown -> markdown_converter.sources (List[Path])
  - text_file_converter.documents -> document_joiner.documents (List[Document])
  - markdown_converter.documents -> document_joiner.documents (List[Document])
  - pypdf_converter.documents -> document_joiner.documents (List[Document])
  - document_joiner.documents -> document_cleaner.documents (List[Document])
  - document_cleaner.documents -> document_splitter.docume

In [8]:
from pathlib import Path
preprocessing_pipeline.run({"file_type_router": {"sources": list(Path(output_dir).glob("**/*"))}})


/Users/ngohongthai/miniconda3/envs/nmt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 1/1 [00:03<00:00,  3.47s/it]


{'document_writer': {'documents_written': 7}}

## Optional) Build a pipeline to query documents

In [9]:
import os
from getpass import getpass

if "HF_API_TOKEN" not in os.environ:
    os.environ["HF_API_TOKEN"] = getpass("Enter Hugging Face token:")


In [10]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceAPIGenerator

template = """
Answer the questions based on the given context.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ question }}
Answer:
"""
pipe = Pipeline()
pipe.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
pipe.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component(
    "llm",
    HuggingFaceAPIGenerator(api_type="serverless_inference_api", api_params={"model": "HuggingFaceH4/zephyr-7b-beta"}),
)

pipe.connect("embedder.embedding", "retriever.query_embedding")
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")


🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceAPIGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [11]:
question = (
    "What ingredients would I need to make vegan keto eggplant lasagna, vegan persimmon flan, and vegan hemp cheese?"
)

pipe.run(
    {
        "embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_new_tokens": 350}},
    }
)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.83s/it]


{'llm': {'replies': ["\n\nVegan Keto Eggplant Lasagna:\n\nIngredients:\n- 2 large eggplants\n- A lot of salt (you should have this in your house already)\n- 1/2 cup store-bought vegan mozzarella (for topping)\n\nPesto:\n- 4 oz basil (generally one large clamshell or 2 small ones)\n- 1/4 cup almonds\n- 1/4 cup nutritional yeast\n- 1/4 cup olive oil\n- 1 recipe vegan pesto (you can find this in the recipe)\n- 1 recipe spinach tofu ricotta (you can find this in the recipe)\n- 1 tsp garlic powder\n- Juice of half a lemon\n- Salt to taste\n\nSpinach Tofu Ricotta:\n- 10 oz firm or extra firm tofu\n- Juice of 1 lemon\n- Garlic powder to taste\n- Salt to taste\n\nInstructions:\n1. Slice the eggplants into 1/4 inch thick slices. Some slices will need to be scrapped because it's difficult to get them all uniformly thin. Use them in soup or something, IDK, man.\n2. Take the eggplant slices and rub both sides with salt. Don't be shy about how much, you're gonna rinse it off anyway.\n3. Put them in